## Embedding explorer

- Embeds a string column using OpenAI Embeddings
- Follows the clustering / projection approach from here: https://umap-learn.readthedocs.io/en/latest/clustering.html

In [ ]:
import pandas
import weave
from weave.legacy.weave.ecosystem import openai
from weave.legacy.weave.ecosystem import umap
from weave.legacy.weave.ecosystem import hdbscan

In [ ]:
#raw_data = pandas.read_csv('/Users/shawn/datasets/wandb_export_2023-06-03T15_01_20.066-07_00.csv')
raw_data = pandas.read_csv('wandb_export_2023-07-25T11_43_18.362-04_00.csv')

#raw_data['request_timestamp'] = pandas.to_datetime(raw_data['request_timestamp'], unit='ms', utc=True)

data = weave.save(weave.legacy.weave.ops.dataframe_to_arrow(raw_data), 'data')
# from weave.legacy.weave.panels_py import panel_autoboard
# panel_autoboard.auto_panels(data)
#data

In [ ]:
data

In [ ]:
weave.legacy.weave.panels.Board(
    vars={
        'data': data,
        'limit': 1005,
        'limited_data': lambda data, limit: data.limit(limit),
        'embeddings': lambda limited_data: openai.openai_embed(limited_data['question'], {"model": "text-embedding-ada-002"}),
        'clusterable_projection': lambda embeddings: umap.umap_project(
            embeddings, {
                'n_neighbors': 30,
                'min_dist': 0,
                'n_components': 2,
            }
        ),
        'clusters': lambda clusterable_projection: hdbscan.hdbscan_cluster(clusterable_projection, {
            'min_samples': 10,
            'min_cluster_size': 50
        }),
        'projection': lambda embeddings: umap.umap_project(embeddings, {})
    },
    panels=[
        weave.legacy.weave.panels.BoardPanel(
            lambda limited_data, projection, clusters: weave.legacy.weave.panels.Plot(
                limited_data,
                x=lambda row, index: projection[index][0],
                y=lambda row, index: projection[index][1],
                label=lambda row, index: clusters[index],
            ),
            id='projection_plot',
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=12)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda projection_plot: projection_plot.selected_data(),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=12)
        )
    ]
)